In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from torch import nn
from datetime import datetime

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

There are a lot of parameters. Ones that might be of interest are marked with `***`.

In [3]:
config = get_config(
    # tokenizer
    tokenizer_path="sentencepiece_bpe.model", # *** path to sentencepiece model
    
    # dataset parameters
    data_path="large_graph",             # *** path to node type
    use_node_types=False,                # node types currently not supported
    use_edge_types=True,                 # whether to use edge types
    filter_edges=None,                   # None or list of edge type names
    self_loops=False,                    # whether to use self loops
    train_frac=0.8,                      # *** fraction of nodes to use for training
    random_seed=42,                      # random seed for splitting dataset int o train test validation
    min_count_for_objectives=5,          # *** minimum frequency of targets
    no_global_edges=False,               # remove global edges
    remove_reverse=False,                # remove reverse edges
    custom_reverse=None,                 # None or list of edges, for which reverse edges should be created (use together with `remove_reverse`)
    partition="large_graph/partition.json.bz2",  # partition into train/test/val
    
    # training parameters
    model_output_dir="large_graph",      # *** directory to save checkpoints and training data
    batch_size=128,                     # *** 
    sampling_neighbourhood_size=10,      # number of dependencies to sample for each node
    neg_sampling_factor=1,               # *** number of negative samples for each positive sample
    epochs=10,                           # *** number of epochs
    elem_emb_size=100,                   # *** dimensionality of target embeddings (for node name prediction)
    pretraining_phase=0,                 # number of epochs for pretraining
    embedding_table_size=200000,         # *** embedding table size for subwords
    save_checkpoints=False,              # set to False if checkpoints are not needed
    save_each_epoch=False,               # save each epoch, useful in case of studying model behavior
    measure_scores=True,                 # *** measure ranking scores during evaluation
    dilate_scores=200,                   # downsampling factor for measuring scores to make evaluation faster
    objectives="node_clf",               # type of objective
    force_w2v_ns=True,                   # negative sampling strategy
    gpu=-1,                              # gpuid
    restore_state=False,
    pretrained=None,

    # model parameters
    node_emb_size=100,                   # *** dimensionality of node embeddings
    h_dim=100,                           # *** should match to node dimensionality
    num_bases=10,                        # number of bases for computing parmetwer weights for different edge types
    dropout=0.2,                         # *** 
    use_self_loop=True,                  #
    activation="tanh",                   # *** 
    learning_rate=1e-3,                  # *** 
    use_gcn_checkpoint=True,
    use_att_checkpoint=True,
    use_gru_checkpoint=True
)

In [4]:
# save_config(config, "config.yaml")

In [5]:
# config = load_config("config.yaml")

In [6]:
config

{'DATASET': {'data_path': 'large_graph',
  'train_frac': 0.8,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': True,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': 42,
  'subgraph_id_column': 'mentioned_in',
  'subgraph_partition': None,
  'partition': 'large_graph/partition.json.bz2'},
 'TRAINING': {'model': 'RGCN',
  'model_output_dir': 'large_graph',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 1,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 10,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'node_clf',
  'save_each_epoch': False,
  'save_checkpoints': False,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': True,
  'use_ns_groups'

# Create Dataset

In [7]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]},
)
ntypes, etypes = dataset.get_graph_types()
config["TRAINING"]['ntypes'] = ntypes
config["TRAINING"]['etypes'] = etypes

# Declare target loading function (labels)

In [8]:
def load_type_prediction():
    return unpersist("large_graph/type_annotations.json.bz2")

# Define objective

Currenlty objectives for node classification (`NodeClassifierObjective`), and name-based node embedding training `SubwordEmbedderObjective`.

![](https://github.com/VitalyRomanov/method-embedding/raw/master/examples/figures/img1.png)


In [9]:
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import ClassifierTargetMapper


class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_node_level_objective(
                objective_name="TypeAnnPrediction",
                objective_class=NodeClassifierObjective,
                label_loader_class=ClassifierTargetMapper,
                dataset=dataset,
                labels_fn=load_type_prediction,  # need to define this function
                tokenizer_path=tokenizer_path,
                masker_fn=dataset.create_subword_masker,
                preload_for="file"
            )
        )

# Run training

In [10]:
# %tensorboard --logdir "large_graph"

In [ ]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    tokenizer_path=config["TOKENIZER"]["tokenizer_path"],
    trainer=Trainer
)